In [11]:
import numpy as np
from scipy.special import erf


class AsianOptionMC_MC(object):
    """ Class for Asian options pricing using control variate
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    
    Unitest (doctest):
    >>> myAsianCall = AsianOptionMC_MC('call', 4., 4., 1., 100., .03, 0, .25, 10000)
    >>> myAsianCall.value
    (0.26141622329842962, 0.25359138249114327, 0.26924106410571597)
    >>> myAsianCall.value_with_control_variate
    (0.25813771282805958, 0.25771678775128265, 0.25855863790483652)
    
    """

    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            self.T = float(T)
            self.M = int(M)
            self.r = float(r)
            self.div = float(div)
            self.sigma = float(sigma)
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Options parameters')

        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(- self.r * self.T)

    @property
    def GeometricAsianOption(self):
        sigsqT = ((self.sigma ** 2 * self.T * (self.M + 1) * (2 * self.M + 1))
                  / (6 * self.M * self.M))
        muT = (0.5 * sigsqT + (self.r - 0.5 * self.sigma ** 2)
               * self.T * (self.M + 1) / (2 * self.M))
        d1 = ((np.log(self.S0 / self.strike) + (muT + 0.5 * sigsqT))
              / np.sqrt(sigsqT))
        d2 = d1 - np.sqrt(sigsqT)
        N1 = 0.5 * (1 + erf(d1 / np.sqrt(2)))
        N2 = 0.5 * (1 + erf(d2 / np.sqrt(2)))
        geometric_value = self.discount * (self.S0 * np.exp(muT) * N1 - self.strike * N2)
        return geometric_value

    @property
    def price_path(self, seed = 100):
        np.random.seed(seed)
        price_path = (self.S0 *
                      np.cumprod (np.exp ((self.r - 0.5 * self.sigma ** 2) * self.time_unit +
                                    self.sigma * np.sqrt(self.time_unit) 
                                          * np.random.randn(self.simulations, self.M)), 1))
        return price_path

    @property
    def MCPayoff(self): 
        if self.option_type == 'call':
            MCpayoff = self.discount \
                              * np.maximum(np.mean(self.price_path,1)-self.strike, 0)
        else:
            MCpayoff = self.discount \
                              * np.maximum(self.strike - np.mean(self.price_path,1), 0)
        return MCpayoff
    
    @property 
    def value(self):
        MCvalue = np.mean(self.MCPayoff)
        MCValue_std = np.std(self.MCPayoff)
        upper_bound = MCvalue + 1.96 * MCValue_std/np.sqrt(self.simulations)
        lower_bound = MCvalue - 1.96 * MCValue_std/np.sqrt(self.simulations)
        return MCvalue, lower_bound, upper_bound
    
    @property
    def value_with_control_variate(self):
        
        geometric_average = np.exp( (1/float(self.M)) * np.sum(np.log(self.price_path), 1) )
        if self.option_type == 'call':
            MCpayoff_geometric = self.discount * np.maximum(geometric_average - self.strike, 0)
        else:
            MCpayoff_geometric = self.discount * np.maximum(self.strike - geometric_average, 0)
        value_with_CV = self.MCPayoff + self.GeometricAsianOption - MCpayoff_geometric        
        value_with_control_variate = np.mean(value_with_CV , 0)
        value_with_control_variate_std = np.std(value_with_CV, 0)
        upper_bound_CV = value_with_control_variate + 1.96 * value_with_control_variate_std/np.sqrt(self.simulations)
        lower_bound_CV = value_with_control_variate - 1.96 * value_with_con
        return value_with_control_variate, lower_bound_CV, upper_bound_CV

In [16]:
myAsianCall = AsianOptionMC_MC('call', 100, 90, 1., 20, .05, 0, .2, 10000)
#myAsianCall.value
myAsianCall.GeometricAsianOption

12.511769674099114

In [17]:
myAsianCall.price_path

array([[ 92.61196039,  94.18335135,  99.31620572, ..., 109.78074063,
        107.81222849, 102.70680508],
       [107.67024259, 115.52834675, 114.40577388, ..., 120.91340933,
        113.96765209, 108.01806779],
       [ 97.74109808,  95.0060504 ,  95.17979627, ...,  80.25550991,
         80.09032785,  77.08163803],
       ...,
       [ 99.67969329,  99.41297511, 101.99846723, ..., 108.36675499,
        106.92473343, 119.28812703],
       [ 98.20816194,  97.29161876,  93.07208788, ..., 102.30911223,
        104.58587084,  95.04753385],
       [108.77030454, 102.01740195,  99.07225241, ...,  89.39081399,
         86.66417562,  90.06160401]])

In [18]:
myAsianCall.MCPayoff

array([11.97223329, 27.91916493,  0.        , ..., 14.71282527,
        5.29406117,  5.66561159])